In [ ]:
import os
import pandas as pd
import numpy as np
import math
import datetime as dt

from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

import seaborn as sns 
import matplotlib.pyplot as plt 

In [ ]:
bitcoindf = pd.read_csv('../input/all-cryptocurrencies-price-20152021/Bitcoin/BTC-USD.csv')
bitcoindf = bitcoindf.rename(columns={'Date': 'date','Open':'open','High':'high','Low':'low','Close':'close',
                                'Adj Close':'adj_close','Volume':'volume'})
bitcoindf.head()

In [ ]:
bitcoindf = bitcoindf.fillna(method = 'ffill')

In [ ]:
bitcoindf['date'] = pd.to_datetime(bitcoindf.date)

In [ ]:
fig = plt.figure(figsize = (15,10))

plt.plot(bitcoindf['date'], bitcoindf['close'], color="red")
plt.title('Bitcoin Close Price')

In [ ]:
lastyear = bitcoindf[bitcoindf['date'] > '09-2020']
lastmonth = bitcoindf[bitcoindf['date'] > '08-2021']

In [ ]:
fig = plt.figure(figsize = (15,15))
plt.plot(lastmonth['date'], lastmonth['close'])
plt.plot(lastmonth['date'], lastmonth['open'])
plt.legend(["Close", "Open"])

plt.show()

In [ ]:
fig = plt.figure(figsize = (15,10))
plt.plot(lastyear['date'], lastyear['close'], color="red")
plt.plot(lastyear['date'], lastyear['open'], color="yellow")
plt.legend(["Close", "Open"])



In [ ]:
fig = plt.figure(figsize = (15,10))

plt.plot(bitcoindf['date'], bitcoindf['close'].rolling(50).mean())
plt.plot(bitcoindf['date'], bitcoindf['close'].rolling(200).mean())
plt.title('Bitcoin Close Price moving average')

In [ ]:
fig = plt.figure(figsize = (15,12))
sns.histplot(bitcoindf['close'],color='darkred', kde=True)
plt.axvline(bitcoindf['close'].mean(), color='k', linestyle='dashed', linewidth=2)
plt.title('Bitcoin Close Price')

In [ ]:
bitcoin_numeric = bitcoindf.select_dtypes(exclude = ["bool"])
bitcoin_numeric.hist(figsize=(18,12))
plt.show()

In [ ]:
closedf = bitcoindf[['date','close']]
print("Shape of close dataframe:", closedf.shape)

In [ ]:
closedf = closedf[closedf['date'] > '2020-09-13']
close_stock = closedf.copy()
print("Total data for prediction: ",closedf.shape[0])

In [ ]:
del closedf['date']
scaler=MinMaxScaler(feature_range=(0,1))
closedf=scaler.fit_transform(np.array(closedf).reshape(-1,1))
print(closedf.shape)

In [ ]:
lastyear2 = bitcoindf[bitcoindf['date'] > '2020-09-13']
bitcoindf2 = lastyear2.drop(['close','adj_close'],axis=1)
bitcoindf2 = bitcoindf2.drop(['date'],axis=1)
bitcoindf2.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(bitcoindf2, closedf, test_size=0.3, random_state=42)

In [ ]:
X_train.head()

In [ ]:
from xgboost import XGBRegressor
my_model = XGBRegressor(n_estimators=1000)
my_model.fit(X_train, y_train, verbose=False)

In [ ]:
predictions = my_model.predict(X_test)
print("Mean Absolute Error - MAE : " + str(mean_absolute_error(y_test, predictions)))
print("Root Mean squared Error - RMSE : " + str(math.sqrt(mean_squared_error(y_test, predictions))))

In [ ]:
train_predict=my_model.predict(X_train)
test_predict=my_model.predict(X_test)

train_predict = train_predict.reshape(-1,1)
test_predict = test_predict.reshape(-1,1)

print("Train data prediction:", train_predict.shape)
print("Test data prediction:", test_predict.shape)

In [ ]:
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 

In [ ]:
fullclose = my_model.predict(bitcoindf2)

In [ ]:
print(len(fullclose))

In [ ]:
fullclose = scaler.inverse_transform(fullclose.reshape(1,-1))


In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(50, 45), dpi=80)
plt.plot(lastyear2['date'], np.transpose(fullclose),color="red")
plt.plot(lastyear2['date'], lastyear2['close'])
plt.show()
